Done By -S.D.SUDHARSAN

Resources : 

https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/

https://learnopencv.com/deep-learning-based-object-detection-using-yolov3-with-opencv-python-c/

https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e





In [36]:
import cv2 as cv
import numpy as np

In [37]:
confThreshold = 0.5
nmsThreshold = 0.5

In [38]:
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [39]:
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    classIds = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        color = (255,255,0)
        if classIds[i] == 0:          # Blue if person
            color = (255,0,0)
        elif classIds[i] == 2:        # Green if car  
            color = (0,255,0)
        elif classIds[i] == 7:        # Red if Truck
            color = (0,0,255)
        
        label = '%s:%s' %  (classes[classIds[i]],round(confidences[i]*100))
        cv.putText(frame,label,(left,top-10),cv.FONT_HERSHEY_SIMPLEX,0.3,color,1)
        cv.rectangle(frame, (left, top), (left+width, top+height),color, 2)

In [40]:
classes = []
with open(r"C:\\Users\\sudha\\Downloads\\archive\\coco.names") as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [41]:
modelConfiguration = "C:\\Users\\sudha\\Downloads\\archive\\yolov3.cfg"
weights = "C:\\Users\\sudha\\Downloads\\archive\\yolov3.weights"
net = cv.dnn.readNetFromDarknet(modelConfiguration,weights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [42]:
cap = cv.VideoCapture("demo_Trim.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    blob = cv.dnn.blobFromImage(frame,1/225,(416,416),[0,0,0],1)
    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))
    postprocess(frame, outs)

    cv.imshow("frame",frame)
    if cv.waitKey(100) == 27:
        break
cap.release()
cv.destroyAllWindows()
    